In [10]:
!pip install keras-tuner
!pip install --upgrade tensorflow



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)
ERROR: No matching distribution found for tensorflow

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [11]:
import pandas as pd
import tensorflow as tf
from constants import target_name, x_path, y_path, x_test_path, y_test_path, preprocessed_data
from sklearn.preprocessing import MinMaxScaler
from dslibs.simple_processing import separate_target, split_train_test

df = pd.read_csv(preprocessed_data)

# scaled = df
scaler = MinMaxScaler()
scaled = scaler.fit_transform(df)
scaled = pd.DataFrame(scaled, columns=df.columns)

length = int(len(scaled) * 0.99)
TEST = scaled[length:]
DATA = scaled[:length]

X, y = separate_target(DATA, target_name)
Xt, yt = separate_target(TEST, target_name)

X.to_csv(x_path, index=False)
y.to_csv(y_path, index=False)
Xt.to_csv(x_test_path, index=False)
yt.to_csv(y_test_path, index=False)


X_train, X_valid, y_train, y_valid = split_train_test(X, y, 0.65)

tf.random.set_seed(7)

X_train.head()

,MSSubClass,LotFrontage,LotArea,Street,Alley,LotShape,LandSlope,OverallQual,OverallCond,YearBuilt,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
1372,0.642857,0.184932,0.039496,1.0,0.5,1.000000,0.0,0.666667,0.625,0.913043,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
8,0.571429,0.102740,0.022529,1.0,0.5,1.000000,0.0,0.666667,0.500,0.427536,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1343,0.571429,0.123288,0.029251,1.0,0.5,1.000000,0.0,0.555556,0.625,0.405797,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1318,0.285714,0.167808,0.063012,1.0,0.5,0.333333,0.0,0.777778,0.500,0.934783,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
610,0.642857,0.167808,0.045572,1.0,0.5,1.000000,0.0,0.888889,0.500,0.927536,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [12]:
import keras_tuner
from dslibs.simple_processing import separate_target, split_train_test

def build_fit_callback(hp):
    split = hp.Float("mp", min_value=0.5, max_value=0.8, step=0.05)
    X_train, X_valid, y_train, y_valid = split_train_test(X, y, split)

    batch_size = 256
    features_count = X_train.shape[1]
    epochs = 60
    optimizer = tf.keras.optimizers.Adam()
    # optimizer = tf.keras.optimizers.Adadelta()
    # optimizer = "adam" #hp.Choice('optimizer', ['adam', 'sgd'])
    # activation = tf.keras.activations.sigmoid()
    activation = tf.keras.activations.relu
    # activation = "relu"#hp.Choice('activation', ['relu', 'sigmoid'])
    mp = 4 #hp.Int("mp", min_value=2, max_value=6, step=2)

    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(features_count, activation=activation, input_shape=[features_count]))
    model.add(tf.keras.layers.Dense(features_count * mp, activation=activation))
    model.add(tf.keras.layers.Dense(features_count * mp, activation=activation))
    model.add(tf.keras.layers.Dense(features_count * mp *2, activation=activation))
    model.add(tf.keras.layers.Dense(features_count * mp, activation=activation))
    model.add(tf.keras.layers.Dense(features_count * mp, activation=activation))
    model.add(tf.keras.layers.Dense(features_count, activation=activation))
    model.add(tf.keras.layers.Dense(1))

    model.compile(
        optimizer=optimizer,
        loss="mean_absolute_error",
        metrics=[tf.keras.metrics.MeanAbsoluteError()]
    )

    model.fit(x=X_train, y=y_train,
                             validation_data=(X_valid, y_valid)
                             , batch_size=batch_size
                             , epochs=epochs
                             # , callbacks=[early_stopping]
                             # , use_multiprocessing=True
                             , verbose='2'
                             # , callbacks=[tensorboard_callback]
                             )

    return model

tuner = keras_tuner.Hyperband(
    hypermodel=build_fit_callback,
    objective=keras_tuner.Objective("val_mean_squared_error", direction="min"),
    # objective=keras_tuner.Objective("val_mean_absolute_error", direction="min"),
    max_epochs=50,
    factor=3,
    seed=42,
    overwrite=True,
    directory="logs/keras_tuner"
)
tuner.search(
    x=X_train,
    y=y_train,
    epochs=10,
    batch_size=256,
    validation_data=(X_valid, y_valid)
)
tuner.results_summary()


Epoch 1/60


2023-03-21 12:38:11.656437: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-21 12:38:11.788552: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at xla_ops.cc:418 : NOT_FOUND: could not find registered platform with id: 0x28a09e860
2023-03-21 12:38:11.788577: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at xla_ops.cc:418 : NOT_FOUND: could not find registered platform with id: 0x28a09e860
2023-03-21 12:38:11.792877: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at xla_ops.cc:418 : NOT_FOUND: could not find registered platform with id: 0x28a09e860
2023-03-21 12:38:11.792895: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at xla_ops.cc:418 : NOT_FOUND: could not find registered platform with id: 0x28a09e860
2023-03-21 12:38:11.797251: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at xla_ops.cc:418 : NOT_FOU

NotFoundError: Graph execution error:

Detected at node 'StatefulPartitionedCall_14' defined at (most recent call last):
    File "/opt/homebrew/Cellar/python@3.10/3.10.10_1/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/opt/homebrew/Cellar/python@3.10/3.10.10_1/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/Users/kkepins-macwro_1/.local/share/virtualenvs/ml-IYlnPhoE/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/Users/kkepins-macwro_1/.local/share/virtualenvs/ml-IYlnPhoE/lib/python3.10/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/Users/kkepins-macwro_1/.local/share/virtualenvs/ml-IYlnPhoE/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 725, in start


In [48]:
winner_params = tuner.get_best_hyperparameters(1)

print("\nWinner Params:\n")
print(winner_params)

winner = tuner.get_best_models()

print("\nWinner Model:\n")
print(winner.evaluate())


Winner Params:

Epoch 1/60


2023-01-13 16:13:27.140760: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Epoch 2/60


2023-01-13 16:13:28.265761: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Epoch 3/60
Epoch 4/60
Epoch 5/60
Epoch 6/60
Epoch 7/60
Epoch 8/60
Epoch 9/60
Epoch 10/60
Epoch 11/60
Epoch 12/60
Epoch 13/60
Epoch 14/60
Epoch 15/60
Epoch 16/60
Epoch 17/60
Epoch 18/60
Epoch 19/60
Epoch 20/60
Epoch 21/60
Epoch 22/60
Epoch 23/60
Epoch 24/60
Epoch 25/60
Epoch 26/60
Epoch 27/60
Epoch 28/60
Epoch 29/60
Epoch 30/60
Epoch 31/60
Epoch 32/60
Epoch 33/60
Epoch 34/60
Epoch 35/60
Epoch 36/60
Epoch 37/60
Epoch 38/60
Epoch 39/60
Epoch 40/60
Epoch 41/60
Epoch 42/60
Epoch 43/60
Epoch 44/60
Epoch 45/60
Epoch 46/60
Epoch 47/60
Epoch 48/60
Epoch 49/60
Epoch 50/60
Epoch 51/60
Epoch 52/60
Epoch 53/60
Epoch 54/60
Epoch 55/60
Epoch 56/60
Epoch 57/60
Epoch 58/60
Epoch 59/60
Epoch 60/60

Winner Model:



In [5]:

features_count = X_train.shape[1]
batch_size = 64
epochs = 100
optimizer = tf.keras.optimizers.Adam()
# optimizer = tf.keras.optimizers.Adamax()
# activation = tf.keras.activations.sigmoid()
activation = tf.keras.activations.relu
mp = 5 #hp.Int("mp", min_value=2, max_value=6, step=2)

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(features_count, activation=activation, input_shape=[features_count]))
model.add(tf.keras.layers.Dense(features_count * mp, activation=activation))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(features_count * mp, activation=activation))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(features_count * mp *2, activation=activation))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(features_count * mp *4, activation=activation))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(features_count * mp *2, activation=activation))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(features_count * mp, activation=activation))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(features_count * mp, activation=activation))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(features_count, activation=activation))
model.add(tf.keras.layers.Dense(1))

model.compile(
    optimizer=optimizer,
    loss="mean_absolute_error",
    metrics=[tf.keras.metrics.MeanAbsoluteError()]
)

history_df = model.fit(x=X_train, y=y_train
          , validation_data=(X_valid, y_valid)
          , batch_size=batch_size
          , epochs=epochs
          # , callbacks=[early_stopping]
          # , use_multiprocessing=True
          , verbose='2'
          # , callbacks=[tensorboard_callback]
          )

score2 = model.evaluate(X_valid, y_valid, verbose=1)
print("Evaluation Score: ", score2)


Epoch 1/100


2023-03-21 12:26:12.846558: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-21 12:26:13.045464: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at xla_ops.cc:418 : NOT_FOUND: could not find registered platform with id: 0x1479a3f20
2023-03-21 12:26:13.045491: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at xla_ops.cc:418 : NOT_FOUND: could not find registered platform with id: 0x1479a3f20
2023-03-21 12:26:13.050354: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at xla_ops.cc:418 : NOT_FOUND: could not find registered platform with id: 0x1479a3f20
2023-03-21 12:26:13.050376: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at xla_ops.cc:418 : NOT_FOUND: could not find registered platform with id: 0x1479a3f20
2023-03-21 12:26:13.065801: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at xla_ops.cc:418 : NOT_FOU

NotFoundError: Graph execution error:

Detected at node 'StatefulPartitionedCall_18' defined at (most recent call last):
    File "/opt/homebrew/Cellar/python@3.10/3.10.10_1/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/opt/homebrew/Cellar/python@3.10/3.10.10_1/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/Users/kkepins-macwro_1/.local/share/virtualenvs/ml-IYlnPhoE/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/Users/kkepins-macwro_1/.local/share/virtualenvs/ml-IYlnPhoE/lib/python3.10/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/Users/kkepins-macwro_1/.local/share/virtualenvs/ml-IYlnPhoE/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 725, in start


In [2]:
import sklearn as skl
from dslibs.simpleplotter import plot_keras_model_history

# score = model.evaluate(X_valid, y_valid, verbose=0)
# print("Loss: {:0.5f}".format(score))

pred = model.predict(Xt)

# uyt = scaler.inverse_transform(yt)
# upred = scaler.inverse_transform(pred)

print('Test Data MAE: ', skl.metrics.mean_absolute_error(yt, pred))
print('Test Data MAE % :', skl.metrics.mean_absolute_percentage_error(yt, pred))

plot_keras_model_history(yt, pred, "baseline", "predictions")


NameError: name 'model' is not defined